# Import von WIAG-Daten in FactGrid
Lies FactGrid-IDs in WIAG ein.  

- Lies die aktuellen FQ-IDs aus WIAG ein. 
- Lies die aktuellen WIAG-IDs aus FactGrid ein.
- Prüfe in Bezug auf die WIAG-ID auf doppelte Einträge in WIAG, in FactGrid und auf unterschiedliche Zuordnungen in WIAG und FactGrid
- Entferne Einträge, die im FactGrid schon eine ID haben.

In [1]:
using DataFrames, CSV

In [2]:
input_path = "C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

In [3]:
output_path = "C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data_sql"

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data_sql"

## Lies und prüfe die IDs aus FactGrid

Frage IDs im FactGrid ab:


``` sparql
SELECT DISTINCT ?item ?itemLabel ?wiag
	WHERE
	{
	  ?item wdt:P601 ?wiag.
	  ?item wdt:P2 wd:Q7
	    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
	}    

```

In [39]:
input_file = "FactGrid-IDs_2023-12-18.tsv"
input_path_file = joinpath(input_path, input_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\FactGrid-IDs_2023-12-18.tsv"

In [40]:
df_fq_ids = CSV.read(input_path_file, DataFrame, delim = "\t");
nrow(df_fq_ids)

5509

In [41]:
df_fq_ids[17:22, :]

Row,item,itemLabel,wiag
,String,String,String31
1,https://database.factgrid.de/entity/Q646336,Johann Adolf Freiherr von Fürstenberg,WIAG-Pers-CANON-10948-001
2,https://database.factgrid.de/entity/Q646337,Adolf Heinrich von Droste zu Vischering,WIAG-Pers-CANON-50828-001
3,https://database.factgrid.de/entity/Q646342,Werner Edler von Plotho,WIAG-Pers-CANON-12623-001
4,https://database.factgrid.de/entity/Q646343,Otto Wackerbarth,WIAG-Pers-CANON-19163-001
5,https://database.factgrid.de/entity/Q646340,Johann Werner von Leerodt,WIAG-Pers-CANON-16673-001
6,https://database.factgrid.de/entity/Q646341,Franz Bernhard Graf von Nassau-Hadamar,WIAG-Pers-CANON-15625-001


In [42]:
find_fq_id(s) = last(split(s, "/"))

find_fq_id (generic function with 1 method)

In [43]:
transform!(df_fq_ids, :item => ByRow(find_fq_id) => :q_id);

Finde doppelte Einträge

In [44]:
df_fq_m = groupby(df_fq_ids, :wiag);
size(df_fq_m)

(5509,)

In [45]:
df_fq_m_count = combine(df_fq_m, nrow => :n);

In [46]:
df_fq_m_2 = subset(df_fq_m_count, :n => ByRow(n -> n > 1))

Row,wiag,n
,String31,Int64


Falls es Mehrfacheinträge gibt: Erhalte mehr Informationen durch einen Filter für diese Einträge

In [21]:
wiag_mult = [
    "WIAG-Pers-CANON-11582-001", 
    "WIAG-Pers-CANON-15754-001"
]

2-element Vector{String}:
 "WIAG-Pers-CANON-11582-001"
 "WIAG-Pers-CANON-15754-001"

In [22]:
crit_wiag(s) = s in wiag_mult
subset(df_fq_ids, :wiag => ByRow(crit_wiag))

Row,item,itemLabel,wiag,factgrid_id
,String,String,String31,SubStrin…
1,https://database.factgrid.de/entity/Q509641,Franz Christoph Anton von Hohenzollern-Sigmaringen,WIAG-Pers-CANON-15754-001,Q509641
2,https://database.factgrid.de/entity/Q509010,Hermann von Baden-Baden,WIAG-Pers-CANON-11582-001,Q509010


Übernimm nur Zeilen ohne die jeweils überzähligen FactGrid-IDs. Die Filterliste enthält IDs, die **nicht** in WIAG übernommen werden sollen.

In [24]:
crit_fq(s) = s in ["Q646386", "Q646394"]

crit_fq (generic function with 1 method)

In [47]:
df_fq_ids = subset(df_fq_ids, :q_id => ByRow(!crit_fq));
nrow(df_fq_ids)

5509

In [47]:
subset(df_fq_ids, :wiag => ByRow(crit_wiag))

Row,item,itemLabel,wiag,factgrid_id
,String,String,String31,SubStrin…
1,https://database.factgrid.de/entity/Q509010,Hermann of Baden-Baden,WIAG-Pers-CANON-11582-001,Q509010
2,https://database.factgrid.de/entity/Q509641,"Francis Christopher Anton, Count of Hohenzollern-Sigmaringen",WIAG-Pers-CANON-15754-001,Q509641


## Lies und prüfe die FactGrid-IDs aus WIAG

Exportiere Personendaten aus WIAG: "Export"/"CSV Personendaten"
Daten: LIVE 2023-12-18 12:20

In [48]:
input_file = "WIAG-can_2023-12-18.csv"
input_path_file = joinpath(input_path, input_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\WIAG-can_2023-12-18.csv"

In [49]:
df_p = CSV.read(input_path_file, DataFrame);
nrow(df_p)

15145

In [50]:
df_p_fq = select(df_p, [:id, :FactGrid_ID, :corpus, :displayname]);

## Vergleiche die beiden Quellen

In [51]:
df_fq = outerjoin(df_p_fq, df_fq_ids, on = :id => :wiag);
nrow(df_fq)

19307

Die Zahl ist größer als die Zahl der Einträge in `df_p_fq` aus WIAG, weil in der Liste aus FactGrid die Bischöfe auch enthalten sind.

In [52]:
names(df_fq)

7-element Vector{String}:
 "id"
 "FactGrid_ID"
 "corpus"
 "displayname"
 "item"
 "itemLabel"
 "q_id"

Einträge mit FactGrid-ID in WIAG aber ohne Eintrag in FactGrid

In [53]:
a_mg_b(a, b) = !ismissing(a) && ismissing(b)

a_mg_b (generic function with 1 method)

In [54]:
df_p_only = subset(df_fq, [:FactGrid_ID, :q_id] => ByRow(a_mg_b));
nrow(df_p_only)

0

Einträge mit unterschiedlichen Werten für die FactGrid-ID

In [55]:
missing_or_equal(a, b) = ismissing(a) || ismissing(b) || a == b

missing_or_equal (generic function with 1 method)

In [57]:
df_diff = subset(df_fq, [:FactGrid_ID, :q_id] => ByRow(!missing_or_equal));
nrow(df_diff)

0

Sortiere die Bischöfe wieder aus. (Nach der Prüfung auf ungleiche IDs!) 

In [75]:
is_gatz_id(s) = occursin("EPISCGatz", s)

is_gatz_id (generic function with 1 method)

In [77]:
subset!(df_fq, :id => ByRow(!is_gatz_id))
nrow(df_fq)

14143

Zahl der neu einzutragenden FactGrid-IDs: Einträge mit einer ID im FactGrid, aber nicht in WIAG

In [79]:
df_insert = subset(df_fq, [:q_id, :FactGrid_ID] => ByRow(a_mg_b));
nrow(df_insert)

82

Insert Statement
``` sql
INSERT INTO url_external (item_id, value, authority_id)
	    SELECT item_id, 'Q648993', 42 FROM item_corpus
	    WHERE id_public = "WIAG-Pers-EPISCGatz-02525-001";
```

In [80]:
function make_insert_stmt(fq_id, wiag_id)
    return """
    INSERT INTO url_external (item_id, value, authority_id)
    SELECT item_id, '$(fq_id)', 42 FROM item_corpus
    WHERE id_public = "$(wiag_id)";
    """
end

make_insert_stmt (generic function with 1 method)

Schreibe eine Datei mit SQL-Statements, mit der die FactGrid IDs in die Tabelle `url_external` eingelesen werden können.

In [81]:
output_file = "insert-uext-can_2023-12-18.sql"
output_file_path = joinpath(output_path, output_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data_sql\\insert-uext-can_2023-12-18.sql"

In [83]:
open(output_file_path, "w") do out
    println(out, "LOCK TABLES url_external WRITE, item_corpus WRITE;")
    for row in eachrow(df_insert)
        println(out, make_insert_stmt(row[:q_id], row[:id]))
    end
    println(out, "UNLOCK TABLES;")
end


Lies die entstandene Datei in die Datenbank ein.